In [ ]:
#Named Entity Recognition using GMB(Groningen Meaning Bank) dataset

In [9]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Download data from kagle

In [ ]:
!pip install kaggle
!pip install pyunpack
!pip install patool

     |████████████████████████████████| 81kB 4.0MB/s 


In [ ]:
#upload the credentials
from google.colab import files
files.upload()

In [ ]:
#The Kaggle API client expects the json file to be in ~/.kaggle folder so let’s create a new folder and move it inside.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=71c10421f520109723da55c9dcb2ea91a306638af5f20436b9f1c1054541f518
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus
#Let’s see the imported files:
!ls

 45% 12.0M/26.4M [00:02<00:04, 3.57MB/s]
100% 26.4M/26.4M [00:03<00:00, 9.16MB/s]
entity-annotated-corpus.zip  gdrive  kaggle.json  sample_data


In [ ]:
#put file name and the dic to store file 
from pyunpack import Archive
Archive('entity-annotated-corpus.zip').extractall('/content/gdrive/MyDrive/NLP projects/NER projects/project 1')

In [ ]:
import pandas as pd
import pandas as pd
df_NER_train = pd.read_csv("/content/gdrive/MyDrive/NLP projects/NER projects/project 1/ner_dataset.csv", encoding="latin1")
df_NER_train.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
import numpy as np
train, validate, test = \
              np.split(df_NER_train.sample(frac=1, random_state=42), 
                       [int(.85*len(df_NER_train)), int(.925*len(df_NER_train))])

In [ ]:
train.head()

,Sentence #,Word,POS,Tag
781974,NaN,the,DT,O
937737,NaN,investors,NNS,O
907828,NaN,New,NNP,B-org
784628,NaN,his,PRP$,O
662460,NaN,the,DT,O


In [ ]:
train.shape,test.shape,validate.shape

((891288, 4), (78644, 4), (78643, 4))

In [ ]:
train= train.reset_index(drop=True)
test= test.reset_index(drop=True)
validate= validate.reset_index(drop=True)

In [ ]:
def flair_datformate(df_NER):
    df_NER=df_NER
    my_dic = {'docs':[], 'tags':[]}
    for i in range(len(df_NER)):
      if df_NER['Word'][i]!='.':
        my_dic['docs'].append(df_NER['Word'][i])
        my_dic['tags'].append(df_NER['Tag'][i])
      else:
        my_dic['docs'].append('.')
        my_dic['tags'].append('.')
        my_dic['docs'].append(' ')
        my_dic['tags'].append(' ')
    return my_dic

In [ ]:
train_dic = flair_datformate(train)
valid_dic = flair_datformate(test)
test_dic = flair_datformate(validate)

In [ ]:
final_train = pd.DataFrame.from_dict(train_dic)
final_valid = pd.DataFrame.from_dict(valid_dic)
final_test = pd.DataFrame.from_dict(test_dic)
final_train[:50]

,docs,tags
0,the,O
1,investors,O
2,New,B-org
3,his,O
4,the,O
5,Earth,B-geo
6,modest,O
7,on,O
8,war,O
9,",",O


In [ ]:
np.savetxt('/content/gdrive/MyDrive/NLP projects/NER projects/project 1/train.txt', final_train.values, fmt='%s', delimiter=" ", header="-DOCSTART- O") 
np.savetxt('/content/gdrive/MyDrive/NLP projects/NER projects/project 1/testa.txt', final_valid.values, fmt='%s', delimiter=" ", header="-DOCSTART- O") 
np.savetxt('/content/gdrive/MyDrive/NLP projects/NER projects/project 1/testb.txt', final_test.values, fmt='%s', delimiter=" ", header="-DOCSTART- O") 

## Flair model

In [10]:
!pip install flair

In [15]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings,StackedEmbeddings
from flair.models import TextClassifier, sequence_tagger_model
from flair.trainers import ModelTrainer
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# tagger = SequenceTagger.load('ner')
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/gdrive/MyDrive/Entity_extarction_aftab/experi-different-model-aftab/NER project 1'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
print(len(corpus.train))

# print the number of Sentences in the test split
print(len(corpus.test))

# print the number of Sentences in the dev split
print(len(corpus.dev))

2021-05-02 11:54:10,356 Reading data from /content/gdrive/MyDrive/Entity_extarction_aftab/experi-different-model-aftab/NER project 1
2021-05-02 11:54:10,357 Train: /content/gdrive/MyDrive/Entity_extarction_aftab/experi-different-model-aftab/NER project 1/train.txt
2021-05-02 11:54:10,359 Dev: /content/gdrive/MyDrive/Entity_extarction_aftab/experi-different-model-aftab/NER project 1/dev.txt
2021-05-02 11:54:10,360 Test: /content/gdrive/MyDrive/Entity_extarction_aftab/experi-different-model-aftab/NER project 1/test.txt
40586
3632
3546


In [16]:
print(corpus.train[0].to_tagged_string('ner'))

# <-DOCSTART-> the investors New <B-org> his the Earth <B-geo> modest on war , officers prompted Sri <B-per> monthly political former Palestinian <B-org> eliminate 60 . <.>


In [17]:
label_dict = corpus.make_label_dictionary()

2021-05-02 11:54:27,601 Computing label dictionary. Progress:


100%|██████████| 44218/44218 [00:01<00:00, 34236.69it/s]

2021-05-02 11:54:28,898 [b'-DOCSTART-', b'O', b'B-org', b'B-geo', b'B-per', b'.', b'B-gpe', b'I-per', b'I-org', b'I-tim', b'B-tim', b'B-art', b'I-geo', b'I-nat', b'B-nat', b'B-eve', b'I-gpe', b'I-art', b'I-eve', b'storm']


In [18]:
label_dict.get_items()

['-DOCSTART-',
 'O',
 'B-org',
 'B-geo',
 'B-per',
 '.',
 'B-gpe',
 'I-per',
 'I-org',
 'I-tim',
 'B-tim',
 'B-art',
 'I-geo',
 'I-nat',
 'B-nat',
 'B-eve',
 'I-gpe',
 'I-art',
 'I-eve',
 'storm']

## Flair with Glove Embedding 

In [ ]:
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove')]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=word_embeddings)
# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=256)
from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)
# 5. create the text classifier
# classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 6. initialize the text classifier trainer
# trainer = ModelTrainer(classifier, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=30)


2021-05-01 09:47:49,881 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp9_jw04ji


100%|██████████| 160000128/160000128 [00:06<00:00, 26193775.98B/s]

2021-05-01 09:47:56,312 copying /tmp/tmp9_jw04ji to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-05-01 09:47:56,922 removing temp file /tmp/tmp9_jw04ji
2021-05-01 09:47:58,642 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpn0f_ckfn


100%|██████████| 21494764/21494764 [00:01<00:00, 15248645.74B/s]

2021-05-01 09:48:00,386 copying /tmp/tmpn0f_ckfn to cache at /root/.flair/embeddings/glove.gensim
2021-05-01 09:48:00,410 removing temp file /tmp/tmpn0f_ckfn


2021-05-01 09:48:04,626 ----------------------------------------------------------------------------------------------------
2021-05-01 09:48:04,628 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=23, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-05-01 09:48:04,631 ----------------------------------------------------------------------------------------------------
2021-05-01 09:48:04,633 Corpus: "Corpus: 40586 train + 3546 dev + 3632 test sentences"
2021-05-01 09:48:04,635 ----------------------------------------------------------------------------------------------------
2021-05-01 09:48:04,637 Parameters:
2021-05-01 09:48:04,638  - learni

{'dev_loss_history': [7.366448879241943,
  6.434582710266113,
  5.904755115509033,
  5.7200212478637695,
  5.672123908996582,
  5.504039287567139,
  5.449540615081787,
  5.392806053161621,
  5.37333345413208,
  5.257050514221191,
  5.211832046508789,
  5.187159538269043,
  5.126097202301025,
  5.1455302238464355,
  5.105213165283203,
  5.104139804840088,
  5.07221794128418,
  5.02623987197876,
  4.989785194396973,
  4.994434356689453,
  4.991807460784912,
  4.927341461181641,
  4.959728240966797,
  4.936580181121826,
  4.910365581512451,
  4.924932479858398,
  4.920487403869629,
  4.901998519897461,
  4.884469509124756,
  4.871624946594238],
 'dev_score_history': [0.7271155869532543,
  0.7566694800190817,
  0.7692031972472132,
  0.7736233854520733,
  0.7746778076759667,
  0.7762205008106646,
  0.7813090779382914,
  0.7829100940793662,
  0.7844533029612757,
  0.7893399223438051,
  0.7894400227401933,
  0.7907108467856131,
  0.7909500986749366,
  0.7877782874078255,
  0.7912803453889165,

In [ ]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')
from flair.data import Sentence
# create example sentence
sentence = Sentence('i am John from lahore')
# predict tags and print
model.predict(sentence)
print(sentence.to_tagged_string())
#print(sentence.labels)
print(sentence)
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

2021-05-01 11:14:33,511 loading file resources/taggers/example-ner/final-model.pt
i am John <B-per> from lahore <B-geo>
Sentence: "i am John from lahore"   [− Tokens: 5  − Token-Labels: "i am John <B-per> from lahore <B-geo>"]
Span [3]: "John"   [− Labels: per (0.6409)]
Span [5]: "lahore"   [− Labels: geo (0.7981)]


In [ ]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')
from flair.data import Sentence
# create example sentence
sentence = Sentence('The first episode aired on 19 November 2004. The show is hosted by producer and director Karan Johar and is produced by SOL India. The show wrapped up its sixth season on 17 March 2019')
# predict tags and print
model.predict(sentence)
print(sentence.to_tagged_string())
#print(sentence.labels)
print(sentence)
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

2021-05-01 11:27:08,961 loading file resources/taggers/example-ner/final-model.pt
The first episode aired on 19 November <B-tim> 2004 <B-tim> . The show is hosted by producer and director Karan <B-per> Johar <B-per> and is produced by SOL <B-per> India <B-geo> . The show wrapped up its sixth season on 17 March <B-tim> 2019 <.>
Sentence: "The first episode aired on 19 November 2004 . The show is hosted by producer and director Karan Johar and is produced by SOL India . The show wrapped up its sixth season on 17 March 2019"   [− Tokens: 37  − Token-Labels: "The first episode aired on 19 November <B-tim> 2004 <B-tim> . The show is hosted by producer and director Karan <B-per> Johar <B-per> and is produced by SOL <B-per> India <B-geo> . The show wrapped up its sixth season on 17 March <B-tim> 2019 <.>"]
Span [7]: "November"   [− Labels: tim (0.8937)]
Span [8]: "2004"   [− Labels: tim (0.5516)]
Span [18]: "Karan"   [− Labels: per (0.6169)]
Span [19]: "Johar"   [− Labels: per (0.2906)]
Span 

## Flair NER model Stacked Embeddings (Glove, forward and backward Flair embeddings)




In [19]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings

tag_type = 'ner'

tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# init standard GloVe embedding
#glove_embedding = WordEmbeddings('glove')

# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

from flair.embeddings import StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
            
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])


from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=stacked_embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)
# 5. create the text classifier
# classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 6. initialize the text classifier trainer
# trainer = ModelTrainer(classifier, corpus)
history=trainer.train('/content/gdrive/MyDrive/NLP projects/NER projects/project 1/',
              learning_rate=0.1,
              mini_batch_size=15,
              max_epochs=20)


2021-05-02 11:54:30,865 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpapuxg7ql


100%|██████████| 73034624/73034624 [00:03<00:00, 18527758.37B/s]

2021-05-02 11:54:35,179 copying /tmp/tmpapuxg7ql to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2021-05-02 11:54:35,244 removing temp file /tmp/tmpapuxg7ql
2021-05-02 11:54:38,907 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpfi5vdjta


100%|██████████| 73034575/73034575 [00:04<00:00, 17441121.34B/s]

2021-05-02 11:54:43,463 copying /tmp/tmpfi5vdjta to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2021-05-02 11:54:43,527 removing temp file /tmp/tmpfi5vdjta
2021-05-02 11:54:44,834 ----------------------------------------------------------------------------------------------------
2021-05-02 11:54:44,836 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_f